In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [3]:
# Cihaz Kontrolü (GPU Kullanımı)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [5]:

# Hiperparametreler
num_epochs = 10
batch_size = 100
learning_rate = 0.001

# MNIST Veri Kümesini Yükleme ve Normalizasyon
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


In [7]:

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████████████████████████████████| 9.91M/9.91M [00:05<00:00, 1.79MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|███████████████████████████████████████| 28.9k/28.9k [00:00<00:00, 159kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████████████████████████████████| 1.65M/1.65M [00:01<00:00, 1.34MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████████████████████████████████| 4.54k/4.54k [00:00<00:00, 2.73MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [9]:
# Lojistik Regresyon Modeli
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
    
    def forward(self, x):
        return self.linear(x)

In [11]:
# Modeli Tanımla ve Cihaza Gönder
input_dim = 28*28  # MNIST görselleri 28x28 boyutundadır
output_dim = 10    # 0-9 arasındaki rakamlar için
model = LogisticRegressionModel(input_dim, output_dim).to(device)

# Kayıp Fonksiyonu ve Optimizasyon
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [13]:
# Model Eğitimi
loss_list = []
iteration_list = []
count = 0

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Görselleri düzleştir ve cihaza (GPU/CPU) gönder
        images = images.view(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Gradients sıfırlama
        optimizer.zero_grad()
        
        # İleri Yayılım
        outputs = model(images)
        
        # Kayıp Hesaplama
        loss = criterion(outputs, labels)
        
        # Geri Yayılım
        loss.backward()
        
        # Parametre Güncelleme
        optimizer.step()
        
        count += 1
        
        # Her 50 iterasyonda bir doğruluk ölç
        if count % 50 == 0:
            correct = 0
            total = 0
            with torch.no_grad():
                for images, labels in test_loader:
                    images = images.view(-1, 28*28).to(device)
                    labels = labels.to(device)

                    outputs = model(images)
                    predicted = torch.argmax(outputs, dim=1)

                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            
            accuracy = 100 * correct / total
            loss_list.append(loss.item())
            iteration_list.append(count)

            if count % 500 == 0:
                print(f'Iteration: {count}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%')

Iteration: 500, Loss: 1.1516, Accuracy: 78.90%
Iteration: 1000, Loss: 0.8488, Accuracy: 83.59%
Iteration: 1500, Loss: 0.8949, Accuracy: 85.27%
Iteration: 2000, Loss: 0.5523, Accuracy: 86.20%
Iteration: 2500, Loss: 0.6297, Accuracy: 86.61%
Iteration: 3000, Loss: 0.5480, Accuracy: 87.23%
Iteration: 3500, Loss: 0.5966, Accuracy: 87.49%
Iteration: 4000, Loss: 0.7496, Accuracy: 87.88%
Iteration: 4500, Loss: 0.5914, Accuracy: 88.23%
Iteration: 5000, Loss: 0.4940, Accuracy: 88.34%
Iteration: 5500, Loss: 0.4545, Accuracy: 88.51%
Iteration: 6000, Loss: 0.4544, Accuracy: 88.76%
